In [1]:
def baseline_gen(signal):
    if len(signal) == 0:
        return 0
    mean = np.mean(signal)
    return mean

def noisy_peak_finding(signal, baseline=1e8):
    
 # Initualize the local peak variable
    peakIndices = []
    peakIndex = None
    peakValue = None
    
    # Find the peak
    for index, value in enumerate(signal):
        if value > baseline:
            if peakValue == None or value > peakValue:
                peakIndex = index
                peakValue = value
        elif value < baseline and peakIndex != None:
            peakIndices.append(peakIndex)
            peakIndex = None
            peakValue = None
            
    if peakIndex != None:
        peakIndices.append(peakIndex)
        
    return peakIndices

def max_peak_finding(signal):
    peakValue = 0
    peakIndex = None
    for index, value in enumerate(signal):
        if value >= peakValue:
            peakIndex = index
            
    return peakIndex

def compute_f0(f_list):
    dif_list = []
    for i in range(len(f_list)-1):
        if f_list[i] == 0:
            continue
        if f_list[i] > 1000:
            continue
        dif=f_list[i+1]-f_list[i]
#         if f_list[i]>250 or f_list[i]<90:
#             continue
        if dif>400 or dif<70:
            continue
        else:
            dif_list.append(dif)
    
    if len(dif_list) == 0:
        return 0
    return dif_list[0]
    
    
def highpass(fre, peak):
    res = []
    for i in range(len(peakList)):
        if frequency[peakList[i]] < 50:
            continue
        res.append(frequency[peakList[i]])
    
    return res
    

In [2]:
# from scipy.signal import butter, lfilter

# def butter_bandpass(lowcut,highcut,fs,order=8):
#     nyq = 0.5*fs
#     low = lowcut/nyq
#     high = highcut/nyq

#     b,a = butter(order, [low, high], btype='band')
#     return b,a

# def butter_bandpass_filter(data,lowcut,highcut,fs,order=8):
#     b,a = butter_bandpass(lowcut,highcut,fs,order=order)
#     return lfilter(b,a,data) 

def find_baseline(data, sampling_rate, OCTAVE):
    i=1
    get_data_slice = data[int((0.5*i-0.5)*WINDOW_SIZE) : int((0.5*i+0.5)*WINDOW_SIZE)]

#     get_data_slice = butter_bandpass_filter(get_data_slice, 50, 400, sampling_rate,order=3)

    fourier_transform = np.fft.rfft(get_data_slice*window, 2**13)

    # To get the power of the fft transform
    abs_fourier_transform = np.abs(fourier_transform)
    power_spectrum = np.square(abs_fourier_transform)

#     plt.plot(power_spectrum)
    down = []
    for i in range(1, OCTAVE+1):
        down.append(power_spectrum[::i])
    #     down1 = power_spectrum[::1]


    res = []
    for i in range(len(down[-1])):
        mul = 1
        for j in range(0, OCTAVE):
            mul *= down[j][i]
        res.append(mul)
    # Normalize the data to get better recognition of the frequency
    # power_spectrum = np.log10(abs_fourier_transform)
    #     print(res)
    maxv = max(res)
    return maxv

In [3]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.io import wavfile

%matplotlib tk

WINDOW_SIZE = 320

f0 = []

SPAN = 0
OCTAVE = 5

sampling_rate, data = wavfile.read('lab_male.wav')

window = np.hanning(WINDOW_SIZE)

# baseline = 1e9
base = find_baseline(data, sampling_rate, OCTAVE)
_ = []

# Based on the symmetry characteristic of signal, we can remove the second half of FFT values and frequency bins
# get only the real fft 

# for i in range(1, int(len(data)/(WINDOW_SIZE))*2-1):
# for j in range(1, 2):
i = 1
# get_data_slice = data[int((0.5*i-0.5)*WINDOW_SIZE) : int((0.5*i+0.5)*WINDOW_SIZE)]
get_data_slice = data[16000: 16320]

#     get_data_slice = butter_bandpass_filter(get_data_slice, 50, 400, sampling_rate,order=3)

fourier_transform = np.fft.rfft(get_data_slice*window, 2**13)
frequency = np.linspace(0, sampling_rate/2, num=len(fourier_transform))
pas = []
for i in range(len(frequency)):
    if frequency[i]<50 or frequency[i]>500:
        pas.append(0)
    else:
        pas.append(1)
for i in range(len(frequency)):
    fourier_transform[i] *= pas[i]
# To get the power of the fft transform
abs_fourier_transform = np.abs(fourier_transform)
power_spectrum = np.square(abs_fourier_transform)


#     plt.plot(power_spectrum)
down = []
for i in range(1, OCTAVE+1):
    down.append(power_spectrum[::i])
#     down1 = power_spectrum[::1]


res = []
for i in range(len(down[-1])):
    mul = 1
    for j in range(0, OCTAVE):
        mul *= down[j][i]
    res.append(mul)
# Normalize the data to get better recognition of the frequency
# power_spectrum = np.log10(abs_fourier_transform)
#     print(res)
maxv = max(res)
indv = res.index(maxv)

if maxv < base:
    indv = 0
    
#     frequency = []
#     for i in range(0, OCTAVE):
#         frequency.append(np.linspace(0, sampling_rate/2, num=len(down[i])))
    #     frequency1 = np.linspace(0, sampling_rate/2, num=len(down1))
    #     plt.plot(frequency1 , down1)



    #     plt.plot(frequency5 , res)


    #     print(indv)
    # #     baseline = baseline_gen(power_spectrum)

    #     peakList = noisy_peak_finding(power_spectrum, baseline)

    
    #     frequency = np.fft.rfftfreq(get_data_slice.size, 1/sampling_rate)

    #     print(frequency)
    #     refindf0 = highpass(frequency, peakList)
if frequency[indv] < 90 or frequency[indv] > 300:
    _.append(0)
#     continue
else:

    _.append(np.mean([frequency[i] for i in range(indv-SPAN, indv+SPAN+1, 1)]))
    print([frequency[i] for i in range(indv-SPAN, indv+SPAN+1, 1)])
    #     f0.append(compute_f0(refindf0)) 

    #     print(refindf0)
    #     print(compute_f0(refindf0))

print(_)
time = []
for i in range(len(_)):
    time.append(int(i*WINDOW_SIZE-WINDOW_SIZE/2))

plt.plot(time, _, 'o')
plt.plot(0, 0, 'o')

[0]


In [4]:


# plt.plot(range(len(power_spectrum)), power_spectrum)

In [5]:
    
time = []
axisf0 = []
for i in range(len(f0)):
    if f0[i] != 0:
        time.append(int(i*WINDOW_SIZE-WINDOW_SIZE/2))
        axisf0.append(f0[i])
    

# plt.plot(time, axisf0, 'o')

In [6]:
len(data)

179985